In [1]:
# NB These are being run as a 40 h, small machine for 1 year ont he NCI supercomputer Gadi.
# Recommend to run with a 40 h machine for headroom, to process 1 year of data. "Small" machine is fine though.
# 24 h was NOT ENOUGH last time.

In [2]:
# V10+ of the retrieval - a clean slate.

import xarray as xr
import matplotlib
matplotlib.use('Agg')   # This helps the PDF output of waveform plots
import matplotlib.pyplot as plt 
import numpy as np
import os
from scipy.signal import argrelextrema
from scipy import stats
import csv
from geopy.distance import geodesic
from IPython.core.debugger import set_trace
import logging
import re
import glob
import pdb

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

os.chdir('/g/data/jk72/af1544/altikamiz/altikamiz')

In [3]:
# set up constants
year = 2024
yearstring = str(year)
version = '0_15'     # V12 has the ice edge slope limit implemented. 
                     # V13 has the reporting of the SIC at the inner miz in the csv.
                     # V14 has the reporting of the coast lat/lon and the "southernmost bit of sea ice" lat/lon in the csv. 
                     # V15 also adds reporting of the lat/lon of where SIC exceeds 80% for the first (northernmost) time.
plotting = False 

In [4]:
# FUNCTION DEFINITIONS!

# Set up logging
logging.basicConfig(filename='./log_output/'+yearstring+'_all_'+version+'.log', level=logging.INFO, format='%(asctime)s - %(message)s')

# Define a custom log function
def log(*args, end='\n', **kwargs):
    message = ' '.join(map(str, args))
    logging.info(message)
    print(message, end=end, **kwargs)  # Optionally print to console

# OK Time to start doing slope of leading edge.
def wf2sle2d(waveform):
     upperthresh=0.8
     lowerthresh=0.2
     # plt.plot(waveform)
     wfmax=waveform.max(dim='range')
     # Find the first index where the threshold is exceeded
     lower_first_exc = (waveform > lowerthresh*wfmax).argmax(dim='range')
     upper_first_exc = (waveform > upperthresh*wfmax).argmax(dim='range')
     # need to watch for run=0. This will manifest as both lower_first_exc and upper_first_exc being the same value.
     # fix this by modifying lower_first_exc to be one lower in this case. It'll be more correctly reported as a high slope, rather than a NaN.
     lower_first_exc = lower_first_exc.where(upper_first_exc > lower_first_exc, lower_first_exc-1)

     run=upper_first_exc-lower_first_exc
     rise=waveform[upper_first_exc]-waveform[lower_first_exc]

     slope=rise/run
     return(slope)

def great_circle_distance(lat1, lon1, lat2, lon2):
    # Create a tuple for each coordinate (latitude, longitude)
    coord1 = (lat1, lon1)
    coord2 = (lat2, lon2)

    # Calculate the great circle distance
    distance = geodesic(coord1, coord2).kilometers

    return distance

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def append_to_csv(file_path, data): 
    # Check if the file exists or is empty to decide whether to write the header row
    file_exists = os.path.isfile(file_path) and os.path.getsize(file_path) > 0

    # Open the CSV file in append mode (a), which allows you to add data without overwriting the existing content
    with open(file_path, mode='a', newline='') as file:
        # Create a CSV writer object
        writer = csv.writer(file)

        # Write the header row if requested and the file is empty or does not exist
        if not file_exists: #write_header and 
            header = ['first_meas_time', 
                      'swhAtMyEdge', 'latAtMyEdge', 'lonAtMyEdge',
                      'latAtAltiKaEdge', 'lonAtAltiKaEdge',
                      'latAtInnerMIZ', 'lonAtInnerMIZ', 'mizWidthAlongTrackFromMyEdge', 'mizWidthAlongTrackFromAltikaEdge',
                      'ice_edge_diff_flag', 'too_many_switches_flag', 'hit_continent_flag', 'inner_miz_sd_km', 
                      'sicedge_lat_p2', 'sicedge_lat_m2', 'ice_edge_complex_flag', 'sic_at_inner_miz',
                      'latAtHitContinentPoint', 'lonAtHitContinentPoint', 'latAtSouthernmostSeaIce', 'lonAtSouthernmostSeaIce',
                      'latAtSICge80', 'lonAtSICge80'
                     ]  
            # With the sd_km variable, it's the inner MIZ location Std.Dev, in km.
            # Here we give 5 thresholds, see when they are exceeded, and see how close these are.
            # If it's insensitive to the choice of threshold then the Std.Dev. will be very low.
            # The idea is to calculate the distance from the AltiKa edge to each inner MIZ, and take the SD of these 5 distances.
                       
            writer.writerow(header)

        # Write the new data to the CSV file
        writer.writerow(data)

def format_longitude(longitude):
    if 0 <= longitude < 180:
        return f"{longitude:.1f}° E"
    elif 180 <= longitude <= 360:
        return f"{360 - longitude:.1f}° W"
    else:
        return "Invalid longitude value"

def format_latitude(latitude):
    if -90 <= latitude < 0:
        return f"{-latitude:.1f}° S"
    elif 0 <= latitude <= 90:
        return f"{latitude:.1f}° N"
    else:
        return "Invalid latitude value"

In [5]:
def multi_panel_plot_with_MIZ_retrieval(ds, latmin, latmax, outfile, plotting):

    try:
        mintime=ds.time.where((ds.open_sea_ice_flag >= 1) & (ds.open_sea_ice_flag <=4) & (ds.lat > latmin) & (ds.lat < latmax), drop=True).min() # this is the earliest time with those conditions
        maxtime=ds.time.where((ds.open_sea_ice_flag >= 1) & (ds.open_sea_ice_flag <=4) & (ds.lat > latmin) & (ds.lat < latmax), drop=True).max() # this is the latest time with those conditions
    except ValueError:
        log("Zero data in mintime or maxtime. Aborting this iteration...", end="")        
        return None

    # subtract 30 s from mintime and add 30 sec to maxtime....
    # This creates a buffer around the times of interest.
    mintime_min30s=mintime - np.timedelta64(40, 's')
    maxtime_pls30s=maxtime + np.timedelta64(40, 's')


    # now for some more panels below this. 
    data = ds['waveforms_40hz'].where((ds.time > mintime_min30s) & (ds.time < maxtime_pls30s), drop=True)
    lat = ds['lat'].where((ds.time > mintime_min30s) & (ds.time < maxtime_pls30s), drop=True)
    range_vals = ds['range']
    peakiness = ds['peakiness_40hz'].where((ds.time > mintime_min30s) & (ds.time < maxtime_pls30s), drop=True)
    sle = ds['sle_40hz'].where((ds.time > mintime_min30s) & (ds.time < maxtime_pls30s), drop=True)
    flag = ds['open_sea_ice_flag'].where((ds.time > mintime_min30s) & (ds.time < maxtime_pls30s), drop=True)
    sic = ds['sic'].where((ds.time > mintime_min30s) & (ds.time < maxtime_pls30s), drop=True)
    # Here are the sic at +2 and -2 degs longitude offset which we will use to assess ice edge morphology
    sicp2 = ds['sicp2'].where((ds.time > mintime_min30s) & (ds.time < maxtime_pls30s), drop=True)
    sicm2 = ds['sicm2'].where((ds.time > mintime_min30s) & (ds.time < maxtime_pls30s), drop=True)


    #########################################################
    # Now try the retrevals.... 
    # first find out which of mintime or maxtime have the lower latitude...
    lat_at_min = ds['lat'].sel(time = mintime, method="nearest")
    lat_at_max = ds['lat'].sel(time = maxtime, method="nearest")

    altika_iceedgelat=np.max([lat_at_min.values, lat_at_max.values])
    altika_iceedgelatplus2=altika_iceedgelat+2

    # Now for inner MIZ...
    sic_thresh = 0.15    
    ice_edge_difference_threshold = 1.0

    # be on the lookout for "in-out-in" - i.e., bands of sea ice.
    # we should use sic here (resampled from the NOAA/NSIDC CDR @ 25 km). 
    sicflag_thresh=(sic >= sic_thresh).astype(int)
    sicflag_thresh_p2=(sicp2 >= sic_thresh).astype(int)
    sicflag_thresh_m2=(sicm2 >= sic_thresh).astype(int) 
    sicflag_thresh_rolmed_1s=((sicflag_thresh).rolling(time=41, center=True)).median()
    sicflag_thresh_rolmed_1s_p2=((sicflag_thresh_p2).rolling(time=41, center=True)).median()
    sicflag_thresh_rolmed_1s_m2=((sicflag_thresh_m2).rolling(time=41, center=True)).median()

    # actually have another threshold for 2 seconds. This is used later.
    sicflag_thresh_rolmed_2s=((sicflag_thresh).rolling(time=81, center=True)).median()   # This one is used for "in out in".

    
    # the first time sicflag_thresh_rolmed_1s goes to 1 is "my" ice edge
    # Do this at +/- 2 deg lon (about 100 km) to assess the ice edge shape:
    # Prefer to only look at "nice" ice edges which don't vary too much in latitude.
    try:
        sicedge_lat=sicflag_thresh_rolmed_1s.lat.where(sicflag_thresh_rolmed_1s > 0.5, drop=True).max()
        sicedge_lat_p2=sicflag_thresh_rolmed_1s.lat.where(sicflag_thresh_rolmed_1s_p2 > 0.5, drop=True).max()
        sicedge_lat_m2=sicflag_thresh_rolmed_1s.lat.where(sicflag_thresh_rolmed_1s_m2 > 0.5, drop=True).max()
    except ValueError:
        log("We have a sic ValueError. Aborting this iteration...", end="")        
        return None
    
    # and reject if these are too different
    ice_edge_diff_flag = False
    if np.abs(sicedge_lat.values - altika_iceedgelat) > ice_edge_difference_threshold:
        log("Ice edge difference between AltiKa's reported edge and that calculated from NOAA CDR sic. We will include this line in output anyway... ", end="")        
        ice_edge_diff_flag = True

    ice_edge_complex_flag = False
    ice_edge_lat_threshold = 1 # degrees
    if (np.abs(sicedge_lat.values - sicedge_lat_p2.values) > ice_edge_lat_threshold) or (np.abs(sicedge_lat.values - sicedge_lat_m2.values) > ice_edge_lat_threshold) or (np.abs(sicedge_lat_m2.values - sicedge_lat_p2.values) > ice_edge_lat_threshold):
        log("Ice edge latitude varies considerably between +/- 2 degrees longitude. We will include this line in output anyway... ", end="")        
        ice_edge_complex_flag = True
        

    smoothing_width=41                             # 41 samples corresponds to a second = ~7 km
    product_threshold=[5400,5700,6000,6300,6600]   # Determined experimentally
    SLE_peakiness_product=sle*peakiness 
    SLE_peakiness_smoothed=SLE_peakiness_product.rolling(time=smoothing_width, center=True).mean() 
    data_array = SLE_peakiness_smoothed.values
    local_maxima_indices = argrelextrema(data_array, np.greater)
    local_maxima_values = data_array[local_maxima_indices]
    local_maxima_data = xr.DataArray(local_maxima_values, dims=['maxima_index'], coords={'maxima_index': local_maxima_indices[0]})
    miz_inner_lat=np.zeros(5)
    for i in range(5):
        local_maxima_data = local_maxima_data.where(local_maxima_data > product_threshold[i]).dropna(dim='maxima_index') 
        try:
            miz_inner_lat_onecandidate=SLE_peakiness_smoothed.lat.isel(time=local_maxima_data.maxima_index[-1]).values
            miz_inner_lat_othercandidate=SLE_peakiness_smoothed.lat.isel(time=local_maxima_data.maxima_index[0]).values
            miz_inner_lat[i]=np.max([miz_inner_lat_onecandidate, miz_inner_lat_othercandidate])
        except IndexError:
            log("IndexError in the MIZ retrieval. Aborting this iteration... ", end="")        
            return None
    if 0 in miz_inner_lat:
        log("Still a 0 in this MIZ candidates array... Aborting...", end="")
        return None

    # for altika's ie, we haven't actually figured out which is further north... yet
    one_altika_ie_candidate=ds.lat.where(ds.time==mintime).mean().values
    other_altika_ie_candidate=ds.lat.where(ds.time==maxtime).mean().values
    lat_ie_altika=np.max([one_altika_ie_candidate, other_altika_ie_candidate])
    
    # subset to between ALTIKA edge and the inner MIZ. Should only have 0 or 1 transitions of myflag_thresh_rolmed_2s. 0 means it's always 1 (ice) and 1 means it goes from 0 (ocean) to 1 (ice) once. 
    # 2 or more means it must drop back to ocean.
    inoutin_subset = sicflag_thresh_rolmed_2s.where((sicflag_thresh_rolmed_2s.lat < lat_ie_altika) & (sicflag_thresh_rolmed_2s.lat > miz_inner_lat[2]), drop=True)
    differences = inoutin_subset.diff(dim='time')
    num_switches = (differences != 0).sum().values
    too_many_switches_flag = False
    if num_switches >= 2:
        log("Ice edge complex - a stretch of ocean between the ice edge and inner MIZ. We will include this line in output anyway... ", end="")        
        too_many_switches_flag = True

    # If there's continent between the AltiKa edge and the inner MIZ then abort.
    hit_continent_flag = False
    inoutin_subset_sic = sic.where((sic.lat < lat_ie_altika) & (sic.lat > miz_inner_lat[2]), drop=True)
    try:
        if np.max(inoutin_subset_sic) > 1.1:
            log("Hit the continent before full attenuation... ", end="")        
            hit_continent_flag = True
    except ValueError:
        log("AltiKa edge north of full attenuation... Aborting...", end="")        
        return None

    # Would also like to know the SIC at the inner edge (miz_inner_lat)
    latitude_diff = abs(sic.lat - miz_inner_lat[2])
    nearest_index = latitude_diff.argmin()
    sic_at_inner_miz = sic.isel(time=nearest_index)

    # Report the lat/lon at both "when we hit the continent" and "the southernmost patch of sea ice" (these can be the same... mostly the same?)
    # For the "southernmost patch of sea ice", we're talking about the lat/lon on the lesser lat of "mintime" and "maxtime"
    altika_southern_iceedge_lat=np.min([lat_at_min.values, lat_at_max.values]) 
    if altika_southern_iceedge_lat == lat_at_min.values:
        altika_southern_iceedge_lon = ds['lon'].sel(time = mintime, method="nearest").values
    else:
        altika_southern_iceedge_lon = ds['lon'].sel(time = maxtime, method="nearest").values

    # Retrieve the northernmost "5" on flag, which is the continent.
    timeOfHitContinent=ds.time.where((ds.open_sea_ice_flag == 5) & (ds.lat > latmin) & (ds.lat < latmax), drop=True).max() 
    otherTimeOfHitContinent=ds.time.where((ds.open_sea_ice_flag == 5) & (ds.lat > latmin) & (ds.lat < latmax), drop=True).min() 

    # Figure out which of these is further north
    latAtOneTimeCont = ds['lat'].sel(time = timeOfHitContinent, method="nearest").values
    latAtOtherTimeCont = ds['lat'].sel(time = otherTimeOfHitContinent, method="nearest").values
    lonAtOneTimeCont = ds['lon'].sel(time = timeOfHitContinent, method="nearest").values
    lonAtOtherTimeCont = ds['lon'].sel(time = otherTimeOfHitContinent, method="nearest").values

    if latAtOneTimeCont > latAtOtherTimeCont:
        lat_at_cont = latAtOneTimeCont
        lon_at_cont = lonAtOneTimeCont
    else:
        lat_at_cont = latAtOtherTimeCont
        lon_at_cont = lonAtOtherTimeCont


    # Need to output the lat/lon of the northernmost point where we exceed 80%.
    # Make sure the transition past the 80% line is calculated properly....
    # Step 1: Sort the DataArray by latitude in descending order
    sorted_sic = sic.sortby('lat', ascending=False)
    # Step 2: Iterate through the sorted array to find the first occurrence where SIC exceeds 0.8
    for i in range(len(sorted_sic)):
        if sorted_sic[i] > 0.8:
            first_exceedance_latitude = sorted_sic.lat[i].item()
            time_index = ds['lat'].sel(time=slice(None)).values == first_exceedance_latitude
            nearest_index = (abs(ds['lat'] - first_exceedance_latitude)).argmin(dim='time')
            first_exceedance_longitude = ds['lon'][nearest_index].values
            break
    else:
        first_exceedance_latitude = None  # Handle case where no value exceeds 0.8
        first_exceedance_longitude = None
        

    #########################################
    # Append to the stats file...
    trackid=ds.attrs['first_meas_time']
    lat_ie_p00=sicedge_lat.values    # this is "my" ice edge (i.e., I calculate it from NOAA CDR). "p00" = plus 0.0 degrees
    lon_ie_p00=ds.lon.where((ds.lat>=(lat_ie_p00-0.1)) & (ds.lat<=(lat_ie_p00+0.1))).mean().values
    swh_ie_p00=ds.swh_40hz.where((ds.lat>=(lat_ie_p00-0.1)) & (ds.lat<=(lat_ie_p00+0.1))).mean().values 
    
    lon_ie_altika=ds.lon.where(ds.lat==lat_ie_altika).mean().values
    lat_innermiz=miz_inner_lat
    lon_innermiz=np.zeros(5)  
    for i in range(5):  # retrieve the lons corresponding to the 5 lats.
        lon_innermiz[i]=ds.lon.where((ds.lat>=(miz_inner_lat[i]-0.1)) & (ds.lat<=(miz_inner_lat[i]+0.1))).mean().values
        
    # Calculate the MIZ widths from "my" ice edge and the AltiKa ice edge.
    mizwidth_myedge=great_circle_distance(lat_innermiz[2], lon_innermiz[2], lat_ie_p00, lon_ie_p00)
    mizwidth_altikaedge=great_circle_distance(lat_innermiz[2], lon_innermiz[2], lat_ie_altika, lon_ie_altika)

    # And to get the SD in MIZ width...
    mizwidth_ens=np.zeros(5)
    for i in range(5):
        mizwidth_ens[i]=great_circle_distance(lat_innermiz[i], lon_innermiz[i], lat_ie_altika, lon_ie_altika)
    mizwidth_SD=np.std(mizwidth_ens)
    
    outdata = [trackid, swh_ie_p00, lat_ie_p00, lon_ie_p00, 
               lat_ie_altika, lon_ie_altika, lat_innermiz[2], lon_innermiz[2], mizwidth_myedge, mizwidth_altikaedge,
              int(ice_edge_diff_flag), int(too_many_switches_flag), int(hit_continent_flag), mizwidth_SD, 
              sicedge_lat_p2.values, sicedge_lat_m2.values, ice_edge_complex_flag, sic_at_inner_miz.values,
              lat_at_cont, lon_at_cont, altika_southern_iceedge_lat, altika_southern_iceedge_lon,
              first_exceedance_latitude, first_exceedance_longitude
              ]

    append_to_csv(outfile, outdata)

    
    # and plot if we're plotting... (slow)
    if(plotting):
        
        # Set the overall size of the figure
        fig = plt.figure(figsize=(30, 20))
        # Create two subplots using gridspec_kw
        gs = fig.add_gridspec(5, 1, height_ratios=[3, 1, 1, 1, 1])

        ######################### 
        # Create the first subplot
        ax1 = fig.add_subplot(gs[0])
        # Set the size of the first subplot
        ax1.set_aspect('auto')  # Adjust the aspect ratio if needed

        ax1.set_xlabel('Latitude, Longitude')
        ax1.set_ylabel('Range')
        ax1.set_title('first_meas_time: '+ds.attrs['first_meas_time'])
        # Plot data in the first subplot
       p1=ax1.pcolormesh(lat.isel(time=slice(None, None, 8)), range_vals, data.isel(time=slice(None, None, 8)))  # factor of 8 subsample really speeds things up.
        
        #add longitude to the xticks
        t,t2=plt.xticks()
        corresponding_lons=data.swap_dims({'time':'lat'}).sel(lat=t, method='nearest').lon.values
        for i in range(7):
            text=t2[i].get_text()
            modified_text = text+' N, {:.2f} E'.format(corresponding_lons[i])
            # now modify t2 with this new text..
            t2[i].set_text(modified_text)
        plt.xticks(ticks=t,labels=t2)

        # Add in the ice edge (from flag) and the continent edge as vertical bars.
        plt.axvline(ds.lat.sel(time=mintime, method='nearest'), color='red', linestyle='--')
        plt.axvline(ds.lat.sel(time=maxtime, method='nearest'), color='red', linestyle='--')
        plt.axvline(sicedge_lat, color='orange', linestyle='--')
        plt.axvline(miz_inner_lat[2], color='pink', linestyle='--')
        plt.xlim(lat.min(), lat.max())

        #########################
        # Create the 2nd subplot - SLE
        ax2 = fig.add_subplot(gs[1])
        ax2.set_aspect('auto')  # Adjust the aspect ratio if needed
        p2 = ax2.plot(lat,sle)
        ax2.set_xlabel('Latitude, Longitude')
        plt.xticks(ticks=t,labels=t2)
        ax2.set_ylabel('SLE')
        plt.xlim(lat.min(), lat.max())
        plt.axvline(ds.lat.sel(time=mintime, method='nearest'), color='red', linestyle='--')
        plt.axvline(ds.lat.sel(time=maxtime, method='nearest'), color='red', linestyle='--')
        plt.axvline(sicedge_lat, color='orange', linestyle='--')
        plt.axvline(miz_inner_lat[2], color='pink', linestyle='--')

        #########################
        # Create the third subplot - SIC
        ax3 = fig.add_subplot(gs[2])
        # Set the size of the second subplot
        ax3.set_aspect('auto')  # Adjust the aspect ratio if needed
        #p3 = ax3.plot(lat,flag)
        p3 = ax3.plot(lat,sic)
        p3 = ax3.plot(lat,sicflag_thresh)
        p3 = ax3.plot(lat,sicflag_thresh_rolmed_1s+0.1)
        p3 = ax3.plot(lat,sicflag_thresh_rolmed_2s+0.2)
        ax3.set_xlabel('Latitude, Longitude')
        plt.xticks(ticks=t,labels=t2)
        ax3.set_ylabel('Flag')
        plt.xlim(lat.min(), lat.max())
        plt.axvline(ds.lat.sel(time=mintime, method='nearest'), color='red', linestyle='--')
        plt.axvline(ds.lat.sel(time=maxtime, method='nearest'), color='red', linestyle='--')
        plt.axvline(sicedge_lat, color='orange', linestyle='--')
        plt.axvline(miz_inner_lat[2], color='pink', linestyle='--')

        #########################
        # Create the 4th subplot - peakiness
        ax4 = fig.add_subplot(gs[3])
        # Set the size of the second subplot
        ax4.set_aspect('auto')  # Adjust the aspect ratio if needed
        p4 = ax4.plot(lat,peakiness)#.rolling(time=41, center=True).mean())
        ax4.set_xlabel('Latitude, Longitude')
        plt.xticks(ticks=t,labels=t2)
        ax4.set_ylabel('Peakiness')
        plt.xlim(lat.min(), lat.max())
        plt.axvline(ds.lat.sel(time=mintime, method='nearest'), color='red', linestyle='--')
        plt.axvline(ds.lat.sel(time=maxtime, method='nearest'), color='red', linestyle='--')
        plt.axvline(sicedge_lat, color='orange', linestyle='--')
        plt.axvline(miz_inner_lat[2], color='pink', linestyle='--')

        #########################
        # Create the 5th subplot - product
        ax5 = fig.add_subplot(gs[4])
        # Set the size of the second subplot
        ax5.set_aspect('auto')  # Adjust the aspect ratio if needed
        p5 = ax5.plot(lat,SLE_peakiness_product)#swh.rolling(time=41, center=True).mean())#.rolling(time=41, center=True).mean())
        p5s = ax5.plot(lat,SLE_peakiness_smoothed)#swh.rolling(time=41, center=True).mean())#.rolling(time=41, center=True).mean())
        ax5.set_xlabel('Latitude, Longitude')
        plt.xticks(ticks=t,labels=t2)
        ax5.set_ylabel('SLE.peakiness product')
        plt.xlim(lat.min(), lat.max())
        plt.axvline(ds.lat.sel(time=mintime, method='nearest'), color='red', linestyle='--')
        plt.axvline(ds.lat.sel(time=maxtime, method='nearest'), color='red', linestyle='--')
        plt.axvline(sicedge_lat, color='orange', linestyle='--')
        plt.axvline(miz_inner_lat[2], color='pink', linestyle='--')

        # save it as PDF...
        plt.savefig('./waveform_output/v'+version+'/AltiKa_output_v'+version+'_'+(ds.attrs['first_meas_time'].replace(" ", "_")).replace(":", "_")+'.pdf', format='pdf')
        plt.close()
 
    return None    


In [ ]:
directory = '/scratch/jk72/af1544/altika_data/'+yearstring+'/'
outfile = './csv_output/'+yearstring+'_all_output_v'+version+'.csv'

# set up static vars
vars_to_keep=['waveforms_40hz', 'open_sea_ice_flag', 'time', 'meas_ind', 'wvf_ind', 'lat', 'lon', 'lat_40hz', 'lon_40hz', 'time_40hz', 'surface_type', 'swh_40hz', 'agc_40hz', 'agc_corr_40hz', 'mean_wave_period_t02', 'mean_wave_direction', 'peakiness_40hz', 'scaling_factor_40hz']
range_index = np.arange(0, 128)

# iterate over files in that directory
for idx, filename in enumerate(sorted(os.listdir(directory))):
    # select a specific file to play with
    #if filename != "SRL_GPS_2PfP133_0142_20190907_154500_20190907_163519.CNES.nc":
    #    continue

    # how to select a range of files (for completing incomplete processing years)
    #if idx <= 7060:
    #    continue

    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        log('Starting '+f+'... ', end='')        

        # Open dataset...
        try:
            dataset=xr.open_dataset(f)
        except ValueError as e:
            # Handle the specific error (e.g., log it, print a warning)
            log('Failed to open '+f+'... ', end='')
            # Continue to the next iteration of the loop
            continue
        keepers=dataset
        keepers = keepers.drop([var for var in dataset.variables if var not in vars_to_keep])
        combined_keepers = keepers.stack(record=("time", "meas_ind"))
        basename=os.path.basename(f)


        # Create the coordinates for the altika data
        coords = {
            'time': combined_keepers.time_40hz.data,
            'lat': xr.DataArray(combined_keepers.lat_40hz.data, dims='time'),
            'lon': xr.DataArray(combined_keepers.lon_40hz.data, dims='time'),
        }

        # Create the xarray dataset
        ds = xr.Dataset(
            {
                'waveforms_40hz': (['range', 'time'], combined_keepers.waveforms_40hz.data),
                'open_sea_ice_flag': (['time'], combined_keepers.open_sea_ice_flag.data),
                'surface_type': (['time'], combined_keepers.surface_type.data),
                'swh_40hz': (['time'], combined_keepers.swh_40hz.data),
                'agc_40hz': (['time'], combined_keepers.agc_40hz.data),
                'agc_corr_40hz': (['time'], combined_keepers.agc_corr_40hz.data),
                'peakiness_40hz': (['time'], combined_keepers.peakiness_40hz.data),
                'scaling_factor_40hz': (['time'], combined_keepers.scaling_factor_40hz.data)
            },
            coords=coords
        )

        dataset.close()
        combined_keepers.close()
        del coords
        
        # Add range as a dimension coordinate
        ds = ds.assign_coords(range=range_index)
        # somehow time has some gaps. The waveform is missing there too. Need to delete these...
        ds = ds.where(~np.isnat(ds.time), np.nan)
        ds = ds.dropna(dim='time')
        ds.attrs['first_meas_time'] = keepers.first_meas_time

        keepers.close()

        # get the SLE
        sle=wf2sle2d(ds.waveforms_40hz)
        # plug it back into the xarray dataset
        ds_with_sle=ds.assign(sle_40hz=sle)

        # open the associated sic along track dataset...
        # Use a regular expression to extract the date-time string 
        match = re.search(r'(\d{8})_(\d{6})', basename)
        if match:
            date_str, time_str = match.groups()
        # Format the date and time strings
        formatted_date = f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:]}"
        formatted_time = f"{time_str[:2]}_{time_str[2:4]}_{time_str[4:]}"
        sicname = f"{formatted_date}_{formatted_time}"
        sicfullname = glob.glob('./sic_retrieval_output/v0_12/'+yearstring+'/Altika_alongtrack_sic_v0_12_'+sicname+'*.nc')
        # open the SIC dataset...
        try:
            ds_sic = xr.open_dataset(sicfullname[0])
        except IndexError:
            log('Missing SIC file I think... Done.', end="")    
            continue

        # Resample SIC to 40 Hz.
        # I can reference it against lat.
        # Create a new DataArray for interpolated sic data, initialised with NaNs
        try: 
            indices = [find_nearest(ds_sic['lat'].values, val) for val in ds_with_sle['lat'].values]
        except ValueError:
            log('ValueError in the resampling SIC to 40 Hz block... Done.', end="")    
            continue
        sic_highres = xr.DataArray(
            [ds_sic['sic'].values[idx] for idx in indices],
            dims=('time',),
            coords={'time': ds_with_sle['time']})
        # Also include the sic at +2 and -2 degrees of longitude.
        sic_highres_p2 = xr.DataArray(
            [ds_sic['sicp2'].values[idx] for idx in indices],
            dims=('time',),
            coords={'time': ds_with_sle['time']})
        sic_highres_m2 = xr.DataArray(
            [ds_sic['sicm2'].values[idx] for idx in indices],
            dims=('time',),
            coords={'time': ds_with_sle['time']})
        # Add SIC to the dataset.
        ds_with_sle['sic'] = sic_highres
        ds_with_sle['sicp2'] = sic_highres_p2
        ds_with_sle['sicm2'] = sic_highres_m2


        ds_sic.close()
        
        #and plot/save!
        multi_panel_plot_with_MIZ_retrieval(ds_with_sle, -80, -55, outfile, plotting)
        ds.close()
        ds_with_sle.close()
        
        log('Done.')    


Starting /scratch/jk72/af1544/altika_data/2024/SRL_GPS_2PfP185_0222_20240903_143728_20240903_152745.CNES.nc... 